In [1]:
import torch
import keras
from sklearn.model_selection import train_test_split

In [2]:
# IMDB 데이터셋
(train_input, train_target), (test_input, test_target) =\
      keras.datasets.imdb.load_data(num_words=500)

# 훈련세트, 검증 세트
train_input, val_input, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [3]:
# 패딩 처리 
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_input, maxlen=100)
val_seq = pad_sequences(val_input, maxlen=100)

In [4]:
train_seq = torch.tensor(train_seq)
val_seq = torch.tensor(val_seq)
train_target = torch.tensor(train_target, dtype=torch.float32)
val_target = torch.tensor(val_target, dtype=torch.float32)

In [5]:
# TensorDataset, DataLoader 
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_seq, train_target)
val_dataset = TensorDataset(val_seq, val_target)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
# 층 구성 
import torch.nn as nn

class IMDBLstm(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(500, 16)
        self.lstm = nn.LSTM(16, 8, num_layers=2, dropout=0.2, batch_first=True)
        self.dense = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        outputs = self.dense(hidden[-1])

        return self.sigmoid(outputs)


In [7]:
from torchinfo import summary
model = IMDBLstm()
summary(model)

Layer (type:depth-idx)                   Param #
IMDBLstm                                 --
├─Embedding: 1-1                         8,000
├─LSTM: 1-2                              1,408
├─Linear: 1-3                            9
├─Sigmoid: 1-4                           --
Total params: 9,417
Trainable params: 9,417
Non-trainable params: 0

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

IMDBLstm(
  (embedding): Embedding(500, 16)
  (lstm): LSTM(16, 8, num_layers=2, batch_first=True, dropout=0.2)
  (dense): Linear(in_features=8, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
# 옵티마이저, 손실함수 
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())